In [2]:
import os
import json
from utils import error_analysis_event

In [5]:
dataset = 'WikiEvents'
best_epochs = {}
for trig_source in ['human', 'keyword', 'llm', 'no_trig']:
    if trig_source == 'no_trig':
        experiment_types = ['no_trig']
    else:
        experiment_types = ['args_only', 'default', 'single_pass']

    for experiment_type in experiment_types:
        scores_dev, scores_test = {}, {}
        for output_dir in os.listdir(f'../experiments/{dataset}/error_analysis/{trig_source}/{experiment_type}'):
            if output_dir != 'plot.png':
                with open(f'../experiments/{dataset}/error_analysis/{trig_source}/{experiment_type}/{output_dir}/dev.json', 'r') as f:
                    dev_outs = json.loads(f.read())
                scores_dev[output_dir] = dev_outs

                with open(f'../experiments/{dataset}/error_analysis/{trig_source}/{experiment_type}/{output_dir}/test.json', 'r') as f:
                    test_outs = json.loads(f.read())
                scores_test[output_dir] = test_outs
        
        best_dev = max(scores_dev.items(), key=lambda info : info[1]['overall_f1'])[0]
        best_test = scores_test[best_dev]
        print(f'{trig_source} {experiment_type}')
        print(best_test)

        if not trig_source in best_epochs:
            best_epochs[trig_source] = {}
        
        best_epochs[trig_source][experiment_type] = best_dev

roles = ['Defeated', 'Investigator', 'Killer', 'Jailer', 'Destroyer', 'ManufacturerAssembler', 'Instrument', 'PlaceOfEmployment', 'Learner', 'Components', 'Vehicle', 'Disabler', 'Injurer', 'Communicator', 'BodyPart', 'Disease', 'Detainee', 'Position', 'Patient', 'PassengerArtifact', 'Defendant', 'Attacker', 'IdentifiedRole', 'Preventer', 'CrashObject', 'Victim', 'Identifier', 'AcquiredEntity', 'Researcher', 'Regulator', 'Observer', 'Artifact', 'Target', 'Participant', 'Recipient', 'Topic', 'Impeder', 'Treater', 'Subject', 'Destination', 'Giver', 'Perpetrator', 'ExplosiveDevice', 'Transporter', 'Employee', 'PaymentBarter', 'Place', 'Damager', 'JudgeCourt', 'ArtifactMoney', 'IdentifiedObject', 'ObservedEntity', 'Demonstrator', 'Victor', 'TeacherTrainer', 'Prosecutor', 'Dismantler', 'DamagerDestroyer', 'Origin']

human args_only
{'overall_f1': 0.5248807089297886, 'overall_precision': 0.77, 'overall_recall': 0.3981385729058945, 'event_type_f1': 0.6810810810810811, 'event_type_precision': 0.9947368421052631, 'event_type_recall': 0.5178082191780822, 'argument_f1': 0.4298245614035088, 'argument_precision': 0.5664739884393064, 'argument_recall': 0.3462897526501767, 'event_type_metrics_per_class': {'Life.Die': {'f1': 0.7777777777777778, 'precision': 1.0, 'recall': 0.6363636363636364}, 'Contact.Contact': {'f1': 0.4324324324324324, 'precision': 1.0, 'recall': 0.27586206896551724}, 'Life.Injure': {'f1': 0.6808510638297872, 'precision': 1.0, 'recall': 0.5161290322580645}, 'Conflict.Attack': {'f1': 0.6979166666666666, 'precision': 0.9852941176470589, 'recall': 0.5403225806451613}, 'Movement.Transportation': {'f1': 0.7272727272727273, 'precision': 1.0, 'recall': 0.5714285714285714}, 'GenericCrime.GenericCrime': {'f1': 1.0, 'precision': 1.0, 'recall': 1.0}, 'Contact.ThreatenCoerce': {'f1': 1.0, 'precision':

In [10]:
def create_directories_for_file(file_path):
    directory_path = os.path.dirname(file_path)
    folders = directory_path.split(os.sep)
    path_to_create = ""
    for folder in folders:
        path_to_create = os.path.join(path_to_create, folder)
        if not os.path.exists(path_to_create):
            os.makedirs(path_to_create)

for split_style in os.listdir('../bucketed/WikiEvent'):
    # for trigger_source in ['keyword', 'human', 'llm']:
    #     for experiment in ['default', 'single_pass', 'args_only']:
    for trigger_source in ['no_trig']:
        for experiment in ['no_trig']:
            best_epoch = best_epochs[trigger_source][experiment]
            best_epoch = best_epoch[best_epoch.index('_') + 1 : ]
            model_out = f'../experiments/{dataset}/g2_environments/{trigger_source}/{experiment}/test_predictions/{best_epoch}.0.txt'
            tanl_ref = f'../data/{dataset}/{trigger_source}/test_event.json'
            # gtt_ref = f'../data/{dataset}/gtt/{trigger_source}/test.json'
            gtt_ref = f'../data/{dataset}/gtt/keyword/test.json'

            with open(f'../bucketed/WikiEvent/{split_style}/test.json', 'r') as f:
                splits = json.loads(f.read())
            
            for split_range, selections in splits.items():
                out_path = f'../ablations/WikiEvent/{split_style}/{split_range}/{trigger_source}/{experiment}.json'
                with open('temp2.json', 'w') as f:
                    f.write(json.dumps(selections))
                create_directories_for_file(out_path)
                error_analysis_event(
                    model_out,
                    tanl_ref,
                    gtt_ref,
                    "hungarian_scorer.py",
                    "types_mapping/types_mapping_wikievents.json",
                    out_path,
                    roles,
                    "scorer_configs/wikievents_config.json",
                    True,
                    'temp2.json'
                )

In [19]:
splits.keys()

dict_keys(['0-1', '2-3', '4-5', '6-7'])

In [20]:
len(splits['6-7'])

2

In [3]:
roles = ['Defeated', 'Investigator', 'Killer', 'Jailer', 'Destroyer', 'ManufacturerAssembler', 'Instrument', 'PlaceOfEmployment', 'Learner', 'Components', 'Vehicle', 'Disabler', 'Injurer', 'Communicator', 'BodyPart', 'Disease', 'Detainee', 'Position', 'Patient', 'PassengerArtifact', 'Defendant', 'Attacker', 'IdentifiedRole', 'Preventer', 'CrashObject', 'Victim', 'Identifier', 'AcquiredEntity', 'Researcher', 'Regulator', 'Observer', 'Artifact', 'Target', 'Participant', 'Recipient', 'Topic', 'Impeder', 'Treater', 'Subject', 'Destination', 'Giver', 'Perpetrator', 'ExplosiveDevice', 'Transporter', 'Employee', 'PaymentBarter', 'Place', 'Damager', 'JudgeCourt', 'ArtifactMoney', 'IdentifiedObject', 'ObservedEntity', 'Demonstrator', 'Victor', 'TeacherTrainer', 'Prosecutor', 'Dismantler', 'DamagerDestroyer', 'Origin']

for trigger_source in ['keyword', 'human', 'llm']:
# for trigger_source in ['human']:
    for experiment in ['default', 'single_pass', 'args_only']:
        model_outs_dev = f'../experiments/WikiEvents/g2_environments/{trigger_source}/{experiment}/dev_predictions'
        model_outs_test = f'../experiments/WikiEvents/g2_environments/{trigger_source}/{experiment}/test_predictions'
        tanl_ref_dev = f'../data/WikiEvents/{trigger_source}/dev_event.json'
        tanl_ref_test = f'../data/WikiEvents/{trigger_source}/test_event.json'
        gtt_ref_dev_og = f'../datasets/WikiEvent/all_roles_in_template/{trigger_source}/dev_trimmed.json'
        gtt_ref_test_og = f'../datasets/WikiEvent/all_roles_in_template/{trigger_source}/test_trimmed.json'

        os.system(f'python3 ../conversion.py --in_file {gtt_ref_dev_og} --out_dev_gtt gtt_ref_dev.json --dataset "WikiEvents"')
        os.system(f'python3 ../conversion.py --in_file {gtt_ref_test_og} --out_test_gtt gtt_ref_test.json --dataset "WikiEvents"')

        dev_scores, test_scores = {}, {}

        for dev_out in os.listdir(model_outs_dev):
            error_analysis_event(
                os.path.join(model_outs_dev, dev_out),
                tanl_ref_dev,
                "gtt_ref_dev.json",
                "hungarian_scorer.py",
                "types_mapping/types_mapping_wikievents.json",
                'json.json',
                roles,
                "scorer_configs/wikievents_config.json",
                True
            )

            with open('json.json', 'r') as f:
                outputs = json.loads(f.read())
            dev_scores[dev_out] = outputs

            error_analysis_event(
                os.path.join(model_outs_test, dev_out),
                tanl_ref_test,
                "gtt_ref_test.json",
                "hungarian_scorer.py",
                "types_mapping/types_mapping_wikievents.json",
                'json.json',
                roles,
                "scorer_configs/wikievents_config.json",
                True
            )
            with open('json.json', 'r') as f:
                outputs = json.loads(f.read())
            test_scores[dev_out] = outputs
        
        best_dev = max(dev_scores.items(), key = lambda info_tup : info_tup[1]['overall_f1'])[0]
        print(model_outs_test)
        print(test_scores[best_dev])

../experiments/WikiEvents/g2_environments/keyword/default/test_predictions
{'overall_f1': 0.3542538354253835, 'overall_precision': 0.3557422969187675, 'overall_recall': 0.3527777777777778, 'event_type_f1': 0.5781250000000001, 'event_type_precision': 0.7302631578947368, 'event_type_recall': 0.47844827586206895, 'argument_f1': 0.0960960960960961, 'argument_precision': 0.8, 'argument_recall': 0.051118210862619806, 'event_type_metrics_per_class': {'Life.Die': {'f1': 0.787878787878788, 'precision': 0.896551724137931, 'recall': 0.7027027027027027}, 'Contact.Contact': {'f1': 0.33333333333333326, 'precision': 0.5555555555555556, 'recall': 0.23809523809523808}, 'Life.Injure': {'f1': 0.5263157894736842, 'precision': 0.9090909090909091, 'recall': 0.37037037037037035}, 'Conflict.Attack': {'f1': 0.675, 'precision': 0.7605633802816901, 'recall': 0.6067415730337079}, 'Movement.Transportation': {'f1': 0.5, 'precision': 1.0, 'recall': 0.3333333333333333}, 'GenericCrime.GenericCrime': {'f1': 0.0, 'preci

In [4]:
roles = ['Defeated', 'Investigator', 'Killer', 'Jailer', 'Destroyer', 'ManufacturerAssembler', 'Instrument', 'PlaceOfEmployment', 'Learner', 'Components', 'Vehicle', 'Disabler', 'Injurer', 'Communicator', 'BodyPart', 'Disease', 'Detainee', 'Position', 'Patient', 'PassengerArtifact', 'Defendant', 'Attacker', 'IdentifiedRole', 'Preventer', 'CrashObject', 'Victim', 'Identifier', 'AcquiredEntity', 'Researcher', 'Regulator', 'Observer', 'Artifact', 'Target', 'Participant', 'Recipient', 'Topic', 'Impeder', 'Treater', 'Subject', 'Destination', 'Giver', 'Perpetrator', 'ExplosiveDevice', 'Transporter', 'Employee', 'PaymentBarter', 'Place', 'Damager', 'JudgeCourt', 'ArtifactMoney', 'IdentifiedObject', 'ObservedEntity', 'Demonstrator', 'Victor', 'TeacherTrainer', 'Prosecutor', 'Dismantler', 'DamagerDestroyer', 'Origin']

for trigger_source in ['no_trig']:
    for experiment in ['no_trig']:
        model_outs_dev = f'../experiments/WikiEvents/g2_environments/{trigger_source}/{experiment}/dev_predictions'
        model_outs_test = f'../experiments/WikiEvents/g2_environments/{trigger_source}/{experiment}/test_predictions'
        tanl_ref_dev = f'../data/WikiEvents/{trigger_source}/dev_event.json'
        tanl_ref_test = f'../data/WikiEvents/{trigger_source}/test_event.json'
        gtt_ref_dev_og = f'../datasets/WikiEvent/all_roles_in_template/human/dev_trimmed.json'
        gtt_ref_test_og = f'../datasets/WikiEvent/all_roles_in_template/human/test_trimmed.json'

        os.system(f'python3 ../conversion.py --in_file {gtt_ref_dev_og} --out_dev_gtt gtt_ref_dev.json --dataset "WikiEvents"')
        os.system(f'python3 ../conversion.py --in_file {gtt_ref_test_og} --out_test_gtt gtt_ref_test.json --dataset "WikiEvents"')

        dev_scores, test_scores = {}, {}

        for dev_out in os.listdir(model_outs_dev):
            error_analysis_event(
                os.path.join(model_outs_dev, dev_out),
                tanl_ref_dev,
                "gtt_ref_dev.json",
                "hungarian_scorer.py",
                "types_mapping/types_mapping_wikievents.json",
                'json.json',
                roles,
                "scorer_configs/wikievents_config.json",
                True
            )

            with open('json.json', 'r') as f:
                outputs = json.loads(f.read())
            dev_scores[dev_out] = outputs

            error_analysis_event(
                os.path.join(model_outs_test, dev_out),
                tanl_ref_test,
                "gtt_ref_test.json",
                "hungarian_scorer.py",
                "types_mapping/types_mapping_wikievents.json",
                'json.json',
                roles,
                "scorer_configs/wikievents_config.json",
                True
            )
            with open('json.json', 'r') as f:
                outputs = json.loads(f.read())
            test_scores[dev_out] = outputs
        
        best_dev = max(dev_scores.items(), key = lambda info_tup : info_tup[1]['overall_f1'])[0]
        print(model_outs_test)
        print(test_scores[best_dev])

types 'attack event' or 'trigger for attack event' not identified
types 'artifact assembled event' or 'trigger for attack event' not identified
types 'spectator for communication event' or 'trigger for communication event' not identified
types 'artifact destroyed event' or 'trigger for attack event' not identified
types 'diner for transportation event' or 'trigger for transportation event' not identified
types 'artifact assembled event' or 'trigger for attack event' not identified
types 'inspector for crime investigation event' or 'trigger for crime investigation event' not identified
types 'artifact destroyed event' or 'trigger for attack event' not identified
types 'detained entity for arrest or jail with detainment event' or 'trigger for arrest or jail with detainment event' not identified
types 'artifact destroyed event' or 'trigger for attack event' not identified
types 'spectator for communication event' or 'trigger for communication event' not identified
types 'inspector for cri